# RAG Chatbot MCP Platform - Exercise Notebook

This notebook exercises the RAG Chatbot MCP Platform by:
1. Loading sample documents
2. Chunking documents
3. Creating embeddings
4. Storing in vector database
5. Retrieving relevant chunks
6. Generating responses using Ollama

## Prerequisites
- **ChromaDB running** (Docker recommended): `docker-compose up -d chromadb`
- **Ollama running locally**: `ollama serve`
- **Smallest working model**: `ollama pull llama3.2:1b` or `ollama pull tinyllama`


In [3]:
# Import required libraries
import sys
from pathlib import Path
import json
import importlib.util

# Add parent directories to path
project_root = Path().absolute().parent.parent
sys.path.insert(0, str(project_root))

# Helper function to load modules with hyphens in path
def load_module_from_path(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

# Check ChromaDB availability
print("Checking ChromaDB availability...")
CHROMADB_AVAILABLE = False
CHROMADB_MODE = "Mock (not available)"

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")  # Suppress pydantic v1 warnings on Python 3.14+
    
    try:
        # Try HttpClient first (for Docker connection)
        try:
            from chromadb import HttpClient
            try:
                # Try to connect to ChromaDB server (Docker)
                test_client = HttpClient(host="localhost", port=8000)
                test_client.list_collections()  # Test connection
                CHROMADB_AVAILABLE = True
                CHROMADB_MODE = "Docker (HTTP)"
                print("✅ ChromaDB server detected at localhost:8000")
            except Exception as e:
                print(f"⚠️ ChromaDB server at localhost:8000 not responding: {e}")
                print("   Make sure Docker container is running: docker-compose up -d chromadb")
                print("   Will use mock mode (limited functionality)")
        except (ImportError, Exception) as e:
            error_msg = str(e)
            if "pydantic" in error_msg.lower() or "ConfigError" in str(type(e).__name__):
                print("⚠️ ChromaDB has compatibility issues with Python 3.14+")
                print("   Note: This is a known issue with pydantic v1")
                print("   Recommendation: Use Python 3.11 or 3.12 for full ChromaDB support")
                print("   Current: Will use mock mode (works for testing)")
            else:
                print(f"⚠️ ChromaDB import failed: {error_msg}")
                print("   Install with: pip install chromadb-client")
            print("   Will use mock mode (limited functionality)")
    except Exception as e:
        print(f"⚠️ Error checking ChromaDB: {e}")
        print("   Will use mock mode (limited functionality)")

# Load MCP server managers
datasources_module = load_module_from_path(
    "datasources_server",
    project_root / "servers" / "mcp-datasources" / "src" / "server.py"
)
chunker_module = load_module_from_path(
    "chunker_server",
    project_root / "servers" / "mcp-chunker" / "src" / "server.py"
)
embeddings_module = load_module_from_path(
    "embeddings_server",
    project_root / "servers" / "mcp-embeddings" / "src" / "server.py"
)
vectorstore_module = load_module_from_path(
    "vectorstore_server",
    project_root / "servers" / "mcp-vectorstore" / "src" / "server.py"
)
retriever_module = load_module_from_path(
    "retriever_server",
    project_root / "servers" / "mcp-retriever" / "src" / "server.py"
)

# Extract managers
DataSourcesManager = datasources_module.DataSourcesManager
ChunkerManager = chunker_module.ChunkerManager
EmbeddingManager = embeddings_module.EmbeddingManager
VectorStoreManager = vectorstore_module.VectorStoreManager
RetrieverManager = retriever_module.RetrieverManager

# Ollama integration
try:
    import ollama
    OLLAMA_AVAILABLE = True
    print("✅ Ollama available")
except ImportError:
    OLLAMA_AVAILABLE = False
    print("⚠️ Ollama not installed. Install with: pip install ollama")

print(f"\n✅ All imports successful!")
print(f"📊 ChromaDB Mode: {CHROMADB_MODE}")


Checking ChromaDB availability...
⚠️ ChromaDB has compatibility issues with Python 3.14+
   Note: This is a known issue with pydantic v1
   Recommendation: Use Python 3.11 or 3.12 for full ChromaDB support
   Current: Will use mock mode (works for testing)
   Will use mock mode (limited functionality)


NameError: name 'logger' is not defined

## Step 1: Initialize Managers


In [ ]:
# Configuration
COLLECTION_NAME = "rag_documents"
EMBEDDING_MODEL = "all-MiniLM-L6-v2"  # Local model
OLLAMA_MODEL = "llama3.2:1b"  # Smallest working model
CHUNK_SIZE = 512
CHUNK_OVERLAP = 50
TOP_K = 3

# Initialize managers
print("Initializing managers...")
datasource_manager = DataSourcesManager()
chunker_manager = ChunkerManager(
    default_chunk_size=CHUNK_SIZE,
    default_overlap=CHUNK_OVERLAP
)
embedding_manager = EmbeddingManager(default_model=EMBEDDING_MODEL)
vectorstore_manager = VectorStoreManager(
    backend="chromadb",
    persist_dir="./chroma_db",
    chroma_host="localhost",
    chroma_port=8000
)
retriever_manager = RetrieverManager()

print("✅ All managers initialized!")


Initializing managers...
✅ All managers initialized!


## Step 2: Load Sample Documents


In [3]:
# Load sample documents
sample_data_path = Path().absolute().parent.parent / "data" / "sample-data"
print(f"Loading documents from: {sample_data_path}")

load_result = datasource_manager.load_files(
    str(sample_data_path),
    pattern="*.md",
    recursive=False
)

if load_result["status"] == "success":
    print(f"✅ Loaded {load_result['count']} documents!")
    loaded_documents = load_result["documents"]
    
    # Display document info
    for i, doc in enumerate(loaded_documents):
        print(f"\nDocument {i+1}:")
        print(f"  Path: {doc.get('path', 'N/A')}")
        print(f"  Type: {doc.get('type', 'N/A')}")
        print(f"  Size: {doc.get('size', 0)} characters")
        print(f"  Preview: {doc.get('content', '')[:100]}...")
else:
    print(f"❌ Error loading documents: {load_result.get('message', 'Unknown error')}")


Loading documents from: C:\Users\learn\Documents\projects\learn-rag-chatbot-mcp\rag-chatbot-mcp\data\sample-data
✅ Loaded 2 documents!

Document 1:
  Path: C:\Users\learn\Documents\projects\learn-rag-chatbot-mcp\rag-chatbot-mcp\data\sample-data\faq.md
  Type: markdown
  Size: 3721 characters
  Preview: # Frequently Asked Questions

## General Questions

### What is the RAG Chatbot MCP Platform?
The RA...

Document 2:
  Path: C:\Users\learn\Documents\projects\learn-rag-chatbot-mcp\rag-chatbot-mcp\data\sample-data\rag-overview.md
  Type: markdown
  Size: 3313 characters
  Preview: # RAG Chatbot Knowledge Base - Sample Document

## Introduction to RAG (Retrieval-Augmented Generati...


## Step 3: Chunk Documents


In [4]:
# Chunk all documents
all_chunks = []
all_metadatas = []
all_ids = []

print("Chunking documents...")
for doc_idx, doc in enumerate(loaded_documents):
    content = doc.get("content", "")
    metadata = doc.get("metadata", {})
    
    # Chunk the document
    chunk_result = chunker_manager.chunk_text(
        text=content,
        strategy="recursive",
        chunk_size=CHUNK_SIZE,
        overlap=CHUNK_OVERLAP
    )
    
    chunks_data = chunk_result.get("chunks", [])
    print(f"  Document {doc_idx+1}: {len(chunks_data)} chunks")
    
    for chunk_idx, chunk_data in enumerate(chunks_data):
        # Extract text from chunk data structure
        chunk_text = chunk_data.get("text", "") if isinstance(chunk_data, dict) else chunk_data
        all_chunks.append(chunk_text)
        all_metadatas.append({
            **metadata,
            "chunk_index": chunk_idx,
            "document_index": doc_idx
        })
        all_ids.append(f"doc_{doc_idx}_chunk_{chunk_idx}")

print(f"\n✅ Created {len(all_chunks)} total chunks from {len(loaded_documents)} documents")
if all_chunks:
    print(f"Sample chunk (first 200 chars): {all_chunks[0][:200]}...")


Chunking documents...
  Document 1: 9 chunks
  Document 2: 10 chunks

✅ Created 19 total chunks from 2 documents
Sample chunk (first 200 chars): # Frequently Asked Questions

## General Questions

### What is the RAG Chatbot MCP Platform?
The RAG Chatbot MCP Platform is a modular system for building retrieval-augmented generation chatbots usin...


## Step 4: Create Embeddings


In [5]:
# Create embeddings for all chunks
print(f"Creating embeddings using model: {EMBEDDING_MODEL}")
print("This may take a moment...")

embed_result = embedding_manager.embed_batch(
    texts=all_chunks,
    model=EMBEDDING_MODEL
)

if embed_result["status"] == "success":
    embeddings = embed_result["embeddings"]
    print(f"✅ Created {len(embeddings)} embeddings")
    print(f"Embedding dimension: {len(embeddings[0])}")
else:
    print(f"❌ Error creating embeddings: {embed_result.get('error', 'Unknown error')}")


Creating embeddings using model: all-MiniLM-L6-v2
This may take a moment...
✅ Created 19 embeddings
Embedding dimension: 384


## Step 5: Store in Vector Database


In [6]:
# Create collection
print(f"Creating collection: {COLLECTION_NAME}")
coll_result = vectorstore_manager.create_collection(
    name=COLLECTION_NAME,
    embedding_dimension=len(embeddings[0]) if embeddings else 384
)
print(f"Collection creation: {coll_result['status']}")

# Add documents to collection
print("Adding documents to vector store...")
add_result = vectorstore_manager.add_documents(
    collection=COLLECTION_NAME,
    documents=all_chunks,
    embeddings=embeddings,
    metadatas=all_metadatas,
    ids=all_ids
)

if add_result["status"] == "success":
    print(f"✅ Indexed {add_result['added']} chunks!")
    
    # Get collection stats
    stats = vectorstore_manager.get_collection_stats(collection=COLLECTION_NAME)
    print(f"\nCollection Stats:")
    print(json.dumps(stats, indent=2))
else:
    print(f"❌ Error indexing: {add_result.get('error', 'Unknown error')}")


Creating collection: rag_documents
Collection creation: success
Adding documents to vector store...
✅ Indexed 19 chunks!

Collection Stats:
{
  "status": "success",
  "collection": "rag_documents",
  "count": 19,
  "mode": "mock"
}


## Step 6: Query the RAG System


In [7]:
# Define a query
query = "What is RAG and how does it work?"

print(f"Query: {query}\n")

# Step 1: Create query embedding
print("1. Creating query embedding...")
query_embed_result = embedding_manager.embed_text(
    text=query,
    model=EMBEDDING_MODEL
)

if query_embed_result["status"] != "success":
    print(f"❌ Error creating query embedding")
else:
    query_embedding = query_embed_result["embedding"]
    print(f"✅ Query embedding created (dimension: {len(query_embedding)})")
    
    # Step 2: Retrieve relevant chunks
    print(f"\n2. Retrieving top {TOP_K} relevant chunks...")
    search_result = vectorstore_manager.search_similar(
        collection=COLLECTION_NAME,
        query_embedding=query_embedding,
        top_k=TOP_K
    )
    
    if search_result["status"] != "success":
        print(f"❌ Error retrieving documents")
    else:
        retrieved_docs = search_result["results"]
        print(f"✅ Retrieved {len(retrieved_docs)} documents\n")
        
        # Display retrieved documents
        print("Retrieved Documents:")
        for i, doc in enumerate(retrieved_docs, 1):
            print(f"\n[{i}] Distance: {doc.get('distance', 0):.4f}")
            print(f"    Text: {doc.get('document', '')[:200]}...")
        
        # Step 3: Format context for generation
        context_docs = []
        for doc in retrieved_docs:
            context_docs.append({
                "text": doc.get("document", ""),
                "metadata": doc.get("metadata", {})
            })
        
        # Step 4: Generate response with Ollama
        print(f"\n3. Generating response using Ollama ({OLLAMA_MODEL})...")
        
        if not OLLAMA_AVAILABLE:
            print("❌ Ollama not available. Install with: pip install ollama")
        else:
            # Format prompt
            context_str = "\n\n".join([
                f"[{i+1}] {doc['text']}" 
                for i, doc in enumerate(context_docs)
            ])
            
            prompt = f"""Based on the following context, answer the question.

Context:
{context_str}

Question: {query}

Answer:"""
            
            try:
                # Call Ollama
                ollama_response = ollama.generate(
                    model=OLLAMA_MODEL,
                    prompt=prompt
                )
                
                response_text = ollama_response["response"]
                print("\n" + "="*60)
                print("RESPONSE:")
                print("="*60)
                print(response_text)
                print("="*60)
                
            except Exception as e:
                print(f"❌ Ollama error: {e}")
                print("Make sure Ollama is running and the model is available.")
                print(f"Try: ollama pull {OLLAMA_MODEL}")


Query: What is RAG and how does it work?

1. Creating query embedding...
✅ Query embedding created (dimension: 384)

2. Retrieving top 3 relevant chunks...
✅ Retrieved 3 documents

Retrieved Documents:

[1] Distance: 0.1000
    Text: # Frequently Asked Questions

## General Questions

### What is the RAG Chatbot MCP Platform?
The RAG Chatbot MCP Platform is a modular system for building retrieval-augmented generation chatbots usin...

[2] Distance: 0.2000
    Text: ### Who should use this platform?
- **AI/ML Engineers** building production RAG systems
- **Developers** exploring MCP and agentic AI
- **Researchers** experimenting with retrieval techniques
- **Stud...

[3] Distance: 0.3000
    Text: ### How do I install the platform?
1. Clone the repository
2. Run `./tests/manual_testing/Solution-Testing/install_dependencies.sh`
3. Configure your environment variables
4. Verify with `./tests/manu...

3. Generating response using Ollama (llama3.2:1b)...


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"



RESPONSE:
RAG stands for Retrieval-Augmented Generation, which refers to a type of artificial intelligence (AI) system that combines the power of retrieval techniques with generation capabilities. In essence, RAG chatbots are designed to learn from a vast amount of text data by first retrieving relevant information and then generating new text based on that knowledge.

The RAG Chatbot MCP Platform is specifically designed for building such systems using the Model Context Protocol (MCP). It provides seven specialized servers that handle different aspects of the RAG pipeline, making it a modular and scalable solution for building retrieval-augmented generation chatbots.


## Step 7: Try Different Queries

You can modify the query in the cell above and re-run it, or create new query cells below.


In [8]:
# Helper function for querying
def query_rag(question, top_k=TOP_K, model=OLLAMA_MODEL):
    """Query the RAG system with a question."""
    print(f"Query: {question}\n")
    
    # Create query embedding
    query_embed_result = embedding_manager.embed_text(
        text=question,
        model=EMBEDDING_MODEL
    )
    
    if query_embed_result["status"] != "success":
        return f"Error creating query embedding"
    
    query_embedding = query_embed_result["embedding"]
    
    # Retrieve relevant chunks
    search_result = vectorstore_manager.search_similar(
        collection=COLLECTION_NAME,
        query_embedding=query_embedding,
        top_k=top_k
    )
    
    if search_result["status"] != "success":
        return f"Error retrieving documents"
    
    retrieved_docs = search_result["results"]
    
    # Format context
    context_str = "\n\n".join([
        f"[{i+1}] {doc.get('document', '')}" 
        for i, doc in enumerate(retrieved_docs)
    ])
    
    # Generate with Ollama
    if not OLLAMA_AVAILABLE:
        return "Ollama not available"
    
    prompt = f"""Based on the following context, answer the question.

Context:
{context_str}

Question: {question}

Answer:"""
    
    try:
        ollama_response = ollama.generate(model=model, prompt=prompt)
        return ollama_response["response"], retrieved_docs
    except Exception as e:
        return f"Error: {e}", []

print("✅ Helper function defined. Use query_rag('your question') to query the system.")


✅ Helper function defined. Use query_rag('your question') to query the system.


In [ ]:
# Example: Try another query
response, sources = query_rag("What are the benefits of RAG?")

print("\n" + "="*60)
print("RESPONSE:")
print("="*60)
print(response)
print("="*60)

print(f"\n📎 Retrieved {len(sources)} source documents")


## Summary

This notebook demonstrated:
1. ✅ Loading documents from the sample data directory
2. ✅ Chunking documents using recursive strategy
3. ✅ Creating embeddings using sentence-transformers
4. ✅ Storing documents in ChromaDB vector store
5. ✅ Retrieving relevant chunks using semantic search
6. ✅ Generating responses using Ollama (smallest working model)

The RAG pipeline is now fully functional! You can:
- Load your own documents
- Experiment with different chunk sizes and strategies
- Try different embedding models
- Query the system with various questions
- Use different Ollama models for generation
